# Трекинг класса человек  

### Вывод информации на экран

In [17]:
from ultralytics import YOLO
import cv2

In [ ]:
# Загрузка добученной модели YOLOv8
model = YOLO("runs/train/yolov10n_bus_stop_finetuned5/weights/best.pt")

In [23]:
# Загрузка изображения
#image = cv2.imread('2.png')
image = cv2.imread('C:/Users/Egor/Desktop/ss/AI-Flow-Detecting/ai-core/dataset/test/images/20250811_221824.jpg')

# Детекция людей (класс '0' в COCO = человек)
results = model(image, classes=[0])  

# Визуализация результатов
annotated_image = results[0].plot()  # Рисует bounding boxes
cv2.imshow('Detected People', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Подсчёт количества людей
people_count = len(results[0].boxes)
print(f"Количество людей на фото: {people_count}")



0: 384x640 (no detections), 9.9ms
Speed: 1.7ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Количество людей на фото: 0


In [13]:
# Загрузка модели YOLOv8 (nano — быстро, подходит для реального времени)
#model = YOLO('yolov10n.pt')

# URL видеопотока (убраны лишние пробелы в конце!)
# https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8
# https://restreamer.vms.evo73.ru/24c3036fe19a150a/stream.m3u8
stream_url = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"

# Открываем видеопоток
cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("❌ Ошибка: Не удалось открыть видеопоток. Проверь URL и соединение.")
    exit()

print("✅ Видеопоток запущен. Детекция людей через YOLOv8...")

while True:
    ret, frame = cap.read()

    if not ret:
        print("⚠️ Не удалось получить кадр. Поток может быть разорван.")
        break

    # Детекция ТОЛЬКО людей (класс 0 в COCO)
    results = model.predict(frame, classes=[0], conf=0.5, verbose=False)

    # Наносим bounding boxes и метки
    annotated_frame = results[0].plot()

    # Подсчёт обнаруженных людей
    people_count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Людей: {people_count}', (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

    # Показываем кадр
    cv2.imshow('YOLOv8 — Детекция людей в реальном времени', annotated_frame)

    # Выход по клавише 'q' или пробелу
    if cv2.waitKey(1) & 0xFF in [ord('q'), ord(' ')]:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
print("⏹️ Работа завершена.")

✅ Видеопоток запущен. Детекция людей через YOLOv8...
⏹️ Работа завершена.


In [10]:
import cv2
from ultralytics import YOLO
import random

def process_video_with_tracking(model, input_source, show_video=True, save_video=False, output_video_path="output_video.mp4"):
    """
    Функция для обработки видео с детекцией объектов и трекингом.

    Аргументы:
    model: Загруженная модель YOLO
    input_source: Путь к видеофайлу или URL видеопотока
    show_video: Показывать ли видео в реальном времени
    save_video: Сохранять ли результат в файл
    output_video_path: Путь для сохранения выходного видео
    """

    # Открыть источник видео (видеофайл или поток)
    cap = cv2.VideoCapture(input_source)

    if not cap.isOpened():
        raise Exception("Ошибка: Не удалось открыть видео или поток.")

    # Получить параметры видео
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Настройка записи видео если нужно сохранять
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()

        if not ret:
            print("⚠️ Не удалось получить кадр. Поток может быть прерван.")
            break

        # Детекция объектов
        results = model.track(
            frame,
            iou=0.4,           # Порог пересечения для трекинга
            conf=0.5,          # Порог уверенности
            persist=True,      # Сохранять треки между кадрами
            imgsz=608,         # Размер изображения для обработки
            verbose=False,     # Не выводить подробную информацию
            tracker="botsort.yaml"  # Тип трекера
        )

        # Наносим bounding boxes и метки
        annotated_frame = results[0].plot()

        # Подсчёт обнаруженных людей (класс 0)
        people_count = len(results[0].boxes)
        cv2.putText(annotated_frame, f'Людей: {people_count}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

        # Если есть трекинг, добавляем ID объектов
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id in zip(boxes, ids):
                # Генерация случайного цвета для каждого объекта на основе его ID
                random.seed(int(id))
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

                # Рисование прямоугольника
                cv2.rectangle(annotated_frame, (box[0], box[1]), (box[2], box[3]), color, 2)
                # Добавление текста с ID
                cv2.putText(
                    annotated_frame,
                    f"Id {id}",
                    (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 255),
                    2,
                )

        # Сохранение кадра в файл если нужно
        if save_video:
            out.write(annotated_frame)

        # Показ кадра если нужно
        if show_video:
            # Уменьшение размера для удобства просмотра
            annotated_frame = cv2.resize(annotated_frame, (0, 0), fx=0.75, fy=0.75)
            cv2.imshow("YOLOv8 — Обнаружение людей в реальном времени", annotated_frame)

        # Выход по нажатию клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Освобождение ресурсов
    cap.release()
    if save_video:
        out.release()

    # Закрытие всех окон OpenCV
    cv2.destroyAllWindows()

# Загрузка модели YOLOv8
model = YOLO('runs/train/yolov10n_bus_stop_finetuned5/weights/best.pt')
model.fuse()  # Оптимизация модели

# Путь к видео или URL потока
# input_source = "test.mp4"  # Локальный файл
input_source = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"  # Видеопоток

# Запуск обработки видео
process_video_with_tracking(
    model,
    input_source,
    show_video=True,
    save_video=False,
    output_video_path="output_video.mp4"
)

YOLOv10m summary (fused): 136 layers, 15,316,063 parameters, 0 gradients, 58.9 GFLOPs
